In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)


In [3]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[ 0.0287693 , -0.34599283]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[-0.56286174, -0.19581768],
        [-0.32133475,  0.26147264],
        [ 0.06642073, -0.35508865],
        [-0.42617822, -0.6157428 ],
        [ 0.18689609,  0.06300543]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [5]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.04406048,  0.13825406],
        [-0.03699161,  0.11225218],
        [-0.15689759,  0.10903784],
        [-0.21763785,  0.07010557],
        [-0.20172308, -0.14788289]],

       [[-0.01717682,  0.02981987],
        [-0.10430572, -0.14269638],
        [-0.19887902, -0.08310509],
        [-0.24058111, -0.04486748],
        [-0.27254787, -0.02406499]],

       [[-0.10616533,  0.0201657 ],
        [-0.18082505,  0.02390315],
        [-0.07671013,  0.02852695],
        [-0.06578913,  0.04143308],
        [-0.19051923,

In [6]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.07831147, -0.0965586 ],
        [-0.01779365, -0.11887628],
        [ 0.08296619,  0.0324672 ],
        [ 0.13574128,  0.14023125],
        [ 0.04207241, -0.06115264]],

       [[-0.10088145, -0.05386702],
        [-0.12852995, -0.17938134],
        [ 0.0374139 , -0.03107864],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.08419396,  0.11892147],
        [ 0.13161962,  0.18480031],
        [ 0.03512407,  0.07090101],
        [-0.08128006,  0.00339743],
        [ 0.        ,

In [7]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.13763377, -0.1295224 ],
        [ 0.21661519, -0.22943629],
        [-0.0023899 , -0.16411231],
        [-0.11471107, -0.16958378],
        [ 0.02585619, -0.25115794]],

       [[ 0.12139569, -0.12941383],
        [ 0.19852839, -0.17285004],
        [-0.0294128 , -0.12356983],
        [-0.12833174, -0.13618571],
        [-0.1775901 , -0.15710598]],

       [[-0.11613604, -0.02125242],
        [-0.17332469, -0.05259616],
        [-0.00435026, -0.19292057],
        [ 0.134812  , -0.2723763 ],
        [-0.04621219,

In [8]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [9]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[  1.29114434e-01,  -5.53925969e-02,  -9.71331820e-02,
           2.30633199e-01,   5.27416915e-02],
        [  2.09479958e-01,   7.84798339e-02,  -1.80883169e-01,
           4.11422282e-01,   6.70683831e-02],
        [  2.22931877e-01,   1.79666623e-01,  -1.46548718e-01,
           4.27704573e-01,   4.46321890e-02],
        [  1.90354943e-01,   1.68069586e-01,  -7.68498629e-02,
           3.71157527e-01,   2.20404081e-02],
        [  1.35643333e-01,   1.28976762e-01,  -2.77136974e-02,
           3.10160309e-01,   9.50391963e-03]],

       [[  8.70940834e-03,   8.56416002e-02,  -2.43809843e-03,
           2.58156240e-01,   7.58125447e-04],
        [  8.35800450e-03,   7.04047829e-02,  -1.11419067e-03,
           2.32453451e-01,   4.86804143e-04],
        [  5.32604242e-03,   5.00893742e-02,  -3.43791384e-04,
           2.02394441e-01,   2.44061419e-04],
        [  2.72247731e-03,   3.52438837e-02,  -9.82648271e-05,
           1.75888419e-01,   1.08418171e-04],
        [  1.2042

In [10]:
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers

    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[-0.00082496,  0.0009439 , -0.00144903, -0.00044591,  0.00163637],
        [-0.00188168,  0.00362622, -0.00558643, -0.00080732,  0.00617222],
        [-0.00253319,  0.00742686, -0.01119375, -0.00075431,  0.01163032],
        [-0.00244941,  0.01149104, -0.01691158, -0.00014369,  0.01617541],
        [-0.00160465,  0.01526388, -0.02194162,  0.00104781,  0.01899436]],

       [[ 0.00088927,  0.00189203, -0.00348615,  0.00105814,  0.00474141],
        [ 0.00197501,  0.00520765, -0.00850258,  0.00265588,  0.01057532],
        [ 0.00309388,  0.00914637, -0.01361109,  0.00458494,  0.0151917 ],
        [ 0.00426535,  0.01311692, -0.01810362,  0.00678184,  0.01779903],
        [ 0.00553434,  0.01676081, -0.02171928,  0.00922502,  0.01844421]],

       [[ 0.00079296,  0.00209456, -0.00358954,  0.00103204,  0.00467386],
        [ 0.00168873,  0.00564453, -0.00863261,  0.0025422 ,  0.01016378],
        [ 0

In [11]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[  1.75447892e-02,   6.63501322e-02,  -9.67777669e-02,
           2.58590840e-03,  -1.54611990e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  2.96115963e-04,   5.80016200e-12,  -2.33135282e-07,
          -3.39965452e-04,  -1.98970991e-03],
        [  3.63272207e-04,   4.52230145e-14,  -3.59546917e-08,
          -2.15122447e-04,  -5.64344868e-04],
        [  3.78486642e-04,   3.51815505e-16,  -4.29459179e-09,
          -9.98409960e-05,  -1.56684793e-04],
        [  0.00000000e+00,   0.00000000e+0

In [12]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ -1.92841843e-01,  -1.34225324e-01,  -2.69000027e-02,
          -1.80554930e-02,   1.94941267e-01],
        [ -7.19250068e-02,  -1.58382058e-01,  -7.01584220e-02,
           1.08926855e-01,   4.64165181e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -1.75887744e-05,  -4.21396876e-03,  -5.69515178e-05,
           3.24945599e-01,   9.61308181e-02],
        [ -2.83591089e-06,  -2.06225691e-03,  -1.12034322e-05,
           3.52701604e-01,   1.07888572e-01],
        [ -3.73815624e-07,  -8.41683534e-04,  -1.81920518e-06,
           3.46305639e-01,   9.06166211e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.

In [13]:
# flattern based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
       [ 42.,  43.,  44.]], dtype=float32)
array([[[   25.,    28.,    31.,    34.,    37.],
        [   70.,    82.,    94.,   106.,   118.],
        [  115.,   136.,   157.,   178

In [14]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.596759


In [19]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)
print("weights : ", weights)
sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Sequence Loss ",
      "\n Loss1: ", sequence_loss1.eval(),
      "\n Loss2: ", sequence_loss2.eval(),
      "\n Loss3: ", sequence_loss3.eval(),
      "\n Loss4: ", sequence_loss4.eval())

weights :  Tensor("Const_26:0", shape=(1, 3), dtype=float32)
Sequence Loss  
 Loss1:  0.513015 
 Loss2:  0.371101 
 Loss3:  1.31326 
 Loss4:  1.31326


In [18]:
weights

<tf.Tensor 'Const_20:0' shape=(1, 3) dtype=float32>